In [ ]:
import numpy as np
import requests
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from collections import Counter
import seaborn as sns
import plotly.express as px
import geopandas as gpd
import folium
from folium.features import Choropleth
import cenpy as c
import networkx as nx
import osmnx as ox
from IPython.display import IFrame
import ast

# Finding insights on AFDC dataset, mostly focusing on EV Chargers

In [ ]:
df = pd.read_csv('data/fuel_stations.csv')
df

There are so many columns in this dataset, lets just look at the ones that could bring more insights on EV Chargers

In [ ]:
df.columns

Below we can see that there are way too many facility types to consider, and many of them are 'nan.' So this isn't very useful to us

In [ ]:
# Count all unique facility in the 'facility_type' column
facility_counts = df['facility_type'].value_counts(dropna=False).to_dict()
facility_counts

It's interesting to see how many are utilizing renewable sources, and theres clearly a very limitted amount, and most are not even documented

In [ ]:
# Count all unique renewable source in the 'ev_renewable_source' column
ev_renewables_count = df['ev_renewable_source'].value_counts(dropna=False).to_dict()
ev_renewables_count

Here we investigate the distribution of the different EV connector types, and there is some interesting data here that tells us that there are far more J1772 plugs than any other charger out there

In [ ]:
# Flatten the lists by parsing the string representation of lists into actual lists
flattened_connectors = []

for connectors in df['ev_connector_types'].dropna():
    # Convert the string representation of lists to actual lists
    connector_list = ast.literal_eval(connectors)
    flattened_connectors.extend(connector_list)

# Count the occurrences of each connector type
connector_counts = Counter(flattened_connectors)

# Include None
nan_count = df['ev_connector_types'].isna().sum()
connector_counts['None'] = nan_count

# Sort the result by count
ev_connectors_count_sorted = dict(sorted(connector_counts.items(), key=lambda item: item[1], reverse=True))

# Display the sorted counts
ev_connectors_count_sorted

Here I plot the number of each connector on a bar graph to get a better visual representation of the differences in values

In [ ]:
plt.figure(figsize=(10,6))
plt.bar(ev_connectors_count_sorted.keys(), ev_connectors_count_sorted.values(), color='skyblue')
plt.title('Count of EV Connector Types in AFDC')
plt.xlabel('Connector Type')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show

Here I get the count of each fuel type found in the dataset, and the overwhelming majority is electric

In [ ]:
# Count all unique fuel types in the 'fuel_type_code' column
fuel_code_count = df['fuel_type_code'].value_counts(dropna=False).to_dict()
fuel_code_count

Here I plot the count of each fuel type to help visualize how drastic the differnece in values are

In [ ]:
plt.figure(figsize=(10,6))
plt.bar(fuel_code_count.keys(), fuel_code_count.values(), color='blue')
plt.title('Count of Fuel Types in AFDC')
plt.xlabel('Fuel Type')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

Here I get the columns I found relevant to get more insight on the EV Chargers, as most of the other columns give us information about other fuel types or are just completely irrelevant.

In [ ]:
# Extracting the relevant columns in the dataframe for analysis
relevant_columns = ['id', 'access_code', 'fuel_type_code', 'open_date', 'date_last_confirmed', 
                    'state', 'zip', 'city', 'latitude', 'longitude', 'ev_connector_types', 'ev_network'
                    ]
rdf = df[relevant_columns]

# Getting only Electric fuel types
rdf_ELEC = rdf[rdf['fuel_type_code'] == 'ELEC']
rdf_ELEC['open_date'] = pd.to_datetime(rdf_ELEC['open_date'], errors='coerce')
rdf_ELEC

Here I use data from the internet that contains all the zip codes in SDGE and I use that to condence down the dataframe into just the rows that have a zip code in the SDGE territory

In [ ]:
# Read in csv that contains all the zip codes in the SDGE Territory 
# and creates a list of unique zip codes in SDGE
SDGE_zip_codes_df = pd.read_csv('data/SDGE_zip_codes_2024.csv')
SDGE_zip_codes = SDGE_zip_codes_df['ZipCode'].unique()
SDGE_zip_codes = [str(zip_code) for zip_code in SDGE_zip_codes]

# Create Dataframe of all Electric Fuel sources in SDGE Zip codes
rdf_ELEC['zip'] = rdf_ELEC['zip'].astype(str)
SDGE_df = rdf_ELEC[rdf_ELEC['zip'].isin(SDGE_zip_codes)]
SDGE_df


# Time series graph shows massive increase in EV Charging Stations in 2020

Getting the total number of EV chargers added per year in the SDGE territory

In [ ]:
# Group by the number entries there are for each year
SDGE_growth_df = SDGE_df.groupby(SDGE_df['open_date'].dt.to_period('Y')).size().reset_index(name='stations_confirmed')

# Create a new column calculating the total number of stations there are
SDGE_growth_df['cumulative_stations'] = SDGE_growth_df['stations_confirmed'].cumsum()
SDGE_growth_df = SDGE_growth_df[(SDGE_growth_df['open_date'].dt.year > 2000) & (SDGE_growth_df['open_date'].dt.year < 2025)]
SDGE_growth_df['open_date'] = SDGE_growth_df['open_date'].dt.to_timestamp()

Plot the Time Series graph to see the massive growth spike in EV chargers being built in 2020

In [ ]:
# Create a plotly express line chart expressing the same data as above but now with interactivity
fig = px.line(
            SDGE_growth_df,
            x='open_date',
            y=['cumulative_stations', 'stations_confirmed'],
            labels={'value': 'Number of Charging Stations',
                    'variable': 'Legend'},
            title='Cumulative Growth of EV Charging Stations in SDGE Territory',
            width=900,
            height=600
            )

# Update the traces for better visibility
fig.update_traces(mode='lines+markers')

# Customize the legend
fig.for_each_trace(lambda t: t.update(name={'cumulative_stations': 'Cumulative Chargers',
                                            'stations_confirmed': 'Chargers Added That Year'}
                                            [t.name]))
fig.show()

Now get the Cumulative Growth of EV Chargering stations in the whole United States

In [ ]:
# Group by the number entries there are for each year
growth_df = rdf_ELEC.groupby(rdf_ELEC['open_date'].dt.to_period('Y')).size().reset_index(name='stations_confirmed')

# Create a new column calculating the total number of stations there are
growth_df['cumulative_stations'] = growth_df['stations_confirmed'].cumsum()
growth_df = growth_df[(growth_df['open_date'].dt.year > 2000) & (growth_df['open_date'].dt.year < 2025)]
growth_df['open_date'] = growth_df['open_date'].dt.to_timestamp()

In [ ]:
# Create a Plotly line chart for Cumulative Growth of EV Charging Stations Over Time in the US
fig = px.line(
    growth_df, 
    x='open_date', 
    y=['cumulative_stations', 'stations_confirmed'],
    labels={'value': 'Number of Charging Stations',
            'variable': 'Legend'},
    title='Cumulative Growth of EV Charging Stations Over Time in the US',
    width=900,  # Set desired width
    height=600   # Set desired height
)

# Update the traces for better visualization
fig.update_traces(mode='lines+markers')

# Customize the legend
fig.for_each_trace(lambda t: t.update(name={'cumulative_stations': 'Cumulative Chargers',
                                            'stations_confirmed': 'Chargers Added That Year'}
                                            [t.name]))
fig.show()

# Time Series Growth of EV Charging Seperated by Ports

In [ ]:
# Explode the lists into individual rows
def safe_literal_eval(val):
    """Safely evaluate strings to Python lists, leaving lists as is."""
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except (ValueError, SyntaxError):
            return val
    return val

# Apply `safe_literal_eval` only to strings
SDGE_df['ev_connector_types'] = SDGE_df['ev_connector_types'].dropna().apply(safe_literal_eval)
SDGE_df_exploded = SDGE_df.explode('ev_connector_types')
SDGE_df_exploded = SDGE_df_exploded.dropna(subset=['ev_connector_types'])

# Remove unwanted connector types (e.g., NEMA types)
SDGE_df_exploded = SDGE_df_exploded[~SDGE_df_exploded['ev_connector_types'].str.contains('NEMA')]

# Group by year and 'ev_connector_types' and count the number of stations confirmed
growth_df = SDGE_df_exploded.groupby([SDGE_df_exploded['open_date'].dt.to_period('Y'), 'ev_connector_types']).size().reset_index(name='stations_confirmed')

# Create cumulative sum for each connector type
growth_df['cumulative_stations'] = growth_df.groupby('ev_connector_types')['stations_confirmed'].cumsum()
growth_df = growth_df[(growth_df['open_date'].dt.year > 2008) & (growth_df['open_date'].dt.year < 2025)]

# Convert 'open_date' to a timestamp for plotting
growth_df['open_date'] = growth_df['open_date'].dt.to_timestamp()

# Create the interactive Plotly plot
fig = px.line(
    growth_df, 
    x='open_date', 
    y='cumulative_stations', 
    color='ev_connector_types',
    labels={'cumulative_stations': 'Number of Charging Stations',
            'open_date': 'Date',
            'ev_connector_types': 'Connector Type'},
    title='Growth of EV Charging Stations by Connector Type Over Time in SDGE Territory',
    width=900,
    height=600
)

fig.show()

In [ ]:

# Apply `safe_literal_eval` only to strings
rdf_ELEC['ev_connector_types'] = rdf_ELEC['ev_connector_types'].dropna().apply(safe_literal_eval)
rdf_ELEC_exploded = rdf_ELEC.explode('ev_connector_types')
rdf_ELEC_exploded = rdf_ELEC_exploded.dropna(subset=['ev_connector_types'])

# Remove unwanted connector types (e.g., NEMA types)
rdf_ELEC_exploded = rdf_ELEC_exploded[~rdf_ELEC_exploded['ev_connector_types'].str.contains('NEMA')]

# Step 2: Group by year and 'ev_connector_types' and count the number of stations confirmed
growth_df = rdf_ELEC_exploded.groupby([rdf_ELEC_exploded['open_date'].dt.to_period('Y'), 'ev_connector_types']).size().reset_index(name='stations_confirmed')

# Step 3: Create cumulative sum for each connector type
growth_df['cumulative_stations'] = growth_df.groupby('ev_connector_types')['stations_confirmed'].cumsum()
growth_df = growth_df[growth_df['open_date'].dt.year > 2008]

# Convert 'open_date' to a timestamp for plotting
growth_df['open_date'] = growth_df['open_date'].dt.to_timestamp()

# Step 4: Create the interactive Plotly plot
fig = px.line(
    growth_df, 
    x='open_date', 
    y='cumulative_stations', 
    color='ev_connector_types',
    labels={'cumulative_stations': 'Number of Charging Stations',
            'open_date': 'Date',
            'ev_connector_types': 'Connector Type'},
    title='Growth of EV Charging Stations by Connector Type Over Time in the United States',
    width=900,
    height=600
)
fig.show()

# Creating Choropleth Geospatial Map of Charger Density in Zip Codes of SDGE

In [ ]:
# Group by 'zip' to get the count of chargers in each ZIP code
chargers_per_zip_SDGE = SDGE_df.groupby('zip').size().reset_index(name='charger_count')
# Load the shapefile containing ZIP code boundaries for all of US
zip_shapefile = gpd.read_file('data/tl_2024_us_zcta520/tl_2024_us_zcta520.shp')
# Ensure that the ZIP code column in both dataframes are of the same type (string)
chargers_per_zip_SDGE['zip'] = chargers_per_zip_SDGE['zip'].astype(str)
zip_shapefile['ZCTA5CE20'] = zip_shapefile['ZCTA5CE20'].astype(str)

# Merge the charger data with the ZIP code shapefile
map_zip_chargers = zip_shapefile.merge(chargers_per_zip_SDGE, left_on='ZCTA5CE20', right_on='zip', how='inner')

# Fill NaN values with 0 for ZIP codes that have no chargers
map_zip_chargers['charger_count'] = map_zip_chargers['charger_count'].fillna(0)
map_zip_chargers = map_zip_chargers.sort_values('charger_count', ascending=False)
map_zip_chargers

Plot the density of EV chargers by zip code on a geospatial map

In [ ]:
# Plot the merged GeoDataFrame
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
map_zip_chargers.plot(column='charger_count', cmap='OrRd', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)

# Add title and axis labels
ax.set_title('EV Charger Density by ZIP Code', fontdict={'fontsize': 15})
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Show the plot
plt.show()

Here I did the same thing but used folium to make the map interactive and more detailed

In [ ]:
# Create a Folium map centered around the region
m = folium.Map(location=[32.8, -117.2], zoom_start=10)  # Center on San Diego area

# Add the GeoJson layer with a choropleth representation
folium.Choropleth(
    geo_data=map_zip_chargers.to_json(),
    data=map_zip_chargers,
    columns=['ZCTA5CE20', 'charger_count'],
    key_on='feature.properties.ZCTA5CE20',
    fill_color='YlOrRd',
    fill_opacity=0.6,
    line_opacity=0.2,
    legend_name='EV Charger Density'
).add_to(m)

# Add tooltips to display ZIP code and charger count when hovering over each area
folium.GeoJson(
    map_zip_chargers.to_json(),
    tooltip=folium.GeoJsonTooltip(fields=['ZCTA5CE20', 'charger_count'],
                                  aliases=['ZIP Code', 'Charger Count'])
).add_to(m)

# Save the map to an HTML file
m.save('figures/ev_charger_density_map.html')

# Display the interactive map
m

# Gathering Census Data with Cenpy and Merging with AFDC for more EDA

In [ ]:
# Use products.APIConnection to connect to the 
# American Community Survey (ACS) dataset 5-Year Estimates
connection = c.products.APIConnection('ACSDT5Y2022')
# Drop all rows where concept is NA
census_df = connection.variables[~connection.variables['concept'].isna()]

# Finding Relationship Between Median Household Income and EV Charger Density

In [ ]:
# MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2021 INFLATION-ADJUSTED DOLLARS)
income_variable = 'B19013_001E'

# Query the data for all zip codes in the ACS 5-Year Estimates dataset
income_data = connection.query(
    cols=[income_variable],
    geo_unit='zip code tabulation area'
)

In [ ]:
# Rename columns and filter for SDGE ZIP codes
income_data = income_data.rename(columns={'zip code tabulation area': 'zip', income_variable: 'median_income'})
SDGE_income_data = income_data[income_data['zip'].isin(chargers_per_zip_SDGE['zip'])]
SDGE_income_data

In [ ]:
# Merge charger count with median income data
chargers_and_income_SDGE = chargers_per_zip_SDGE.merge(SDGE_income_data, on='zip', how='left')

# Merge the combined data with the ZIP code shapefile
map_chargers_income = zip_shapefile.merge(chargers_and_income_SDGE, left_on='ZCTA5CE20', right_on='zip', how='inner')
map_chargers_income['charger_count'] = pd.to_numeric(map_chargers_income['charger_count'], errors='coerce').fillna(0)
map_chargers_income['median_income'] = pd.to_numeric(map_chargers_income['median_income'], errors='coerce').fillna(0)

Created a dataframe that has zip code, charger count, and median income that can then be plotted on a map

In [ ]:
# Drop rows that didn't have a valid median_income 
map_chargers_income = map_chargers_income[map_chargers_income['median_income'] >= 0]
map_chargers_income.sort_values(by='median_income', ascending=False)

In [ ]:
# Create a Folium map centered around the San Diego area
m = folium.Map(location=[32.8, -117.2], zoom_start=10)

# Add a Choropleth layer for EV Charger Density
folium.Choropleth(
    geo_data=map_chargers_income.to_json(),
    data=map_chargers_income,
    columns=['ZCTA5CE20', 'charger_count'],
    key_on='feature.properties.ZCTA5CE20',
    fill_color='YlOrRd',
    fill_opacity=0.5,
    line_opacity=0.2,
    legend_name='EV Charger Density'
).add_to(m)

# Add another Choropleth layer for Median Income
folium.Choropleth(
    geo_data=map_chargers_income.to_json(),
    data=map_chargers_income,
    columns=['ZCTA5CE20', 'median_income'],
    key_on='feature.properties.ZCTA5CE20',
    fill_color='BuPu',
    fill_opacity=0.5,
    line_opacity=0.2,
    legend_name='Median Household Income'
).add_to(m)

# Add tooltips to display ZIP code, charger count, and median income
folium.GeoJson(
    map_chargers_income.to_json(),
    tooltip=folium.GeoJsonTooltip(
        fields=['ZCTA5CE20', 'charger_count', 'median_income'],
        aliases=['ZIP Code', 'Charger Count', 'Median Income']
    )
).add_to(m)

# Save the map to an HTML file
m.save('figures/ev_charger_income_density_map.html')

# Display the interactive map
m

In [ ]:
fig = px.scatter(
    map_chargers_income,
    x='median_income',
    y='charger_count',
    labels={'median_income': 'Median Household Income', 'charger_count': 'Charger Count'},
    title='Relationship Between Median Household Income and EV Charger Count by Zip Code in SDGE'
)
fig.show()

# Finding Relationship between Vehicle Counts and EV Charger Density in SDGE zip codes 

In [ ]:
# Define vehicle ownership variables, between 1 vehicle households and 3 vehicle households inclusive
vehicle_vars = ['B25044_004E', 'B25044_005E', 'B25044_006E']  

# Query for these variables across ZIP codes
vehicle_data = connection.query(
    cols=vehicle_vars,
    geo_unit='zip code tabulation area'
)

# Ensure columns are numeric after querying
vehicle_data['B25044_004E'] = pd.to_numeric(vehicle_data['B25044_004E'], errors='coerce')
vehicle_data['B25044_005E'] = pd.to_numeric(vehicle_data['B25044_005E'], errors='coerce')
vehicle_data['B25044_006E'] = pd.to_numeric(vehicle_data['B25044_006E'], errors='coerce')

# Rename columns for clarity
vehicle_data = vehicle_data.rename(columns={
    'B25044_004E': 'households_1_vehicle',
    'B25044_005E': 'households_2_vehicles',
    'B25044_006E': 'households_3_or_more_vehicles'
})
# Estimate total vehicles (approximate by using 3 vehicles for "3 or more" category)
vehicle_data['total_vehicles'] = (
    vehicle_data['households_1_vehicle'] * 1 +
    vehicle_data['households_2_vehicles'] * 2 +
    vehicle_data['households_3_or_more_vehicles'] * 3
)

# Calculate vehicle ownership rate (vehicles per household)
vehicle_data['vehicle_per_household'] = vehicle_data['total_vehicles'] / (
    vehicle_data['households_1_vehicle'] +
    vehicle_data['households_2_vehicles'] +
    vehicle_data['households_3_or_more_vehicles']
)

In [ ]:
vehicle_data = vehicle_data.rename(columns={'zip code tabulation area': 'zip'})
vehicle_data_SDGE = vehicle_data[vehicle_data['zip'].isin(chargers_per_zip_SDGE['zip'])]
vehicle_data_SDGE = vehicle_data_SDGE.dropna()
vehicle_data_SDGE['zip'] = vehicle_data_SDGE['zip'].astype(str)

# Merge charger count with vehicle data
chargers_and_vehicles_SDGE = chargers_per_zip_SDGE.merge(vehicle_data_SDGE, on='zip', how='inner')

# Merge the vehicle data with the ZIP code shapefile on 'zip'
map_chargers_vehicles = zip_shapefile.merge(chargers_and_vehicles_SDGE, left_on='ZCTA5CE20', right_on='zip', how='inner')

In [ ]:
# Create a Folium map centered around the San Diego area
m = folium.Map(location=[32.8, -117.2], zoom_start=10)

folium.Choropleth(
    geo_data=map_chargers_vehicles.to_json(),
    data=map_chargers_vehicles,
    columns=['ZCTA5CE20', 'charger_count'],
    key_on='feature.properties.ZCTA5CE20',
    fill_color='YlOrRd',
    fill_opacity=0.5,
    line_opacity=0.2,
    legend_name='EV Charger Density'
).add_to(m)


# Add a choropleth layer to represent the total vehicles per ZIP code
folium.Choropleth(
    geo_data=map_chargers_vehicles.to_json(),
    data=map_chargers_vehicles,
    columns=['ZCTA5CE20', 'total_vehicles'],
    key_on='feature.properties.ZCTA5CE20',
    fill_color='YlGnBu',
    fill_opacity=0.5,
    line_opacity=0.2,
    legend_name='Total Vehicles per ZIP Code'
).add_to(m)

# Add tooltips to display ZIP code and total vehicles when hovering over each area
folium.GeoJson(
    map_chargers_vehicles.to_json(),
    tooltip=folium.GeoJsonTooltip(
        fields=['ZCTA5CE20', 'charger_count', 'total_vehicles'],
        aliases=['ZIP Code', 'Charger Count', 'Total Vehicles']
    )
).add_to(m)

# Save the map to an HTML file
m.save('figures/vehicle_density_map.html')

# Display the interactive map
m

In [ ]:
fig = px.scatter(
    map_chargers_vehicles,
    x='total_vehicles',
    y='charger_count',
    labels={'total_vehicles': 'Total Vehicles', 'charger_count': 'Charger Count'},
    title='Relationship Between Total Vehicles and EV Charger Count by Zip Code in SDGE'
)
fig.show()

# Using osmnx to Find the Distance Between My EV Charger and SDGE

In [ ]:
# Defined starting and ending points (latitude, longitude)
point_a = (32.87506710695114, -117.20368651139246)  # My Charger Location
point_b = (32.82432124022306, -117.14287218508292)  # SDGE Century Park Location

# Download the network graph for the area around the points
G = ox.graph_from_point(point_a, dist=7000, network_type='drive')  # Increase dist if needed to cover the area

# Get the nearest nodes on the network to your points
node_a = ox.distance.nearest_nodes(G, X=point_a[1], Y=point_a[0])
node_b = ox.distance.nearest_nodes(G, X=point_b[1], Y=point_b[0])

# Calculate the shortest path length (in meters) between the nodes
distance_meters = nx.shortest_path_length(G, node_a, node_b, weight='length')
print(f"The shortest path distance between the points is {distance_meters:.2f} meters.")

# Calculate the shortest path between the nodes (in terms of network distance)
shortest_path = nx.shortest_path(G, node_a, node_b, weight='length')

# Get the bounding box for the route (slightly expanded)
north, south, east, west = ox.utils_geo.bbox_from_point(point_a, dist=7000)

# Plot the graph and highlight the shortest path
fig, ax = ox.plot_graph_route(
    G, 
    route=shortest_path, 
    route_linewidth=4, 
    node_size=0, 
    bgcolor='w',
    show=False,
    close=False,
    bbox=(north, south, east, west)  # Set custom bounding box
)

# Add starting and ending points to the plot
ax.plot(point_a[1], point_a[0], marker='o', markersize=10, color='red', label='Start')
ax.plot(point_b[1], point_b[0], marker='o', markersize=10, color='blue', label='End')

# Add legend and show the plot
plt.legend()
plt.title("Shortest Path between Two Points")
plt.show()

In [ ]:
# Extract latitude and longitude of each node in the shortest path
route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]

# Create a Folium map centered between the two points
map_center = ((point_a[0] + point_b[0]) / 2, (point_a[1] + point_b[1]) / 2)
m = folium.Map(location=map_center, zoom_start=13)

# Add the shortest path as a Polyline on the map
folium.PolyLine(route_coords, color='red', weight=5, opacity=0.7).add_to(m)

# Add markers for the start and end points
folium.Marker(location=point_a, tooltip='Start', icon=folium.Icon(color='green')).add_to(m)
folium.Marker(location=point_b, tooltip='End', icon=folium.Icon(color='blue')).add_to(m)

# Add a legend manually using a HTML/CSS-based approach
legend_html = '''
     <div style="position: fixed; 
                 bottom: 20px; left: 20px; width: 140px; height: 110px; 
                 background-color: white; border:2px solid grey; z-index:9999; font-size:14px;">
     &nbsp; <b>Legend</b> <br>
     &nbsp; Start Point &nbsp; <i class="fa fa-map-marker fa-2x" style="color:green"></i><br>
     &nbsp; End Point &nbsp; <i class="fa fa-map-marker fa-2x" style="color:blue"></i><br>
     &nbsp; Route &nbsp; <i style="color:red;">&#8212;&#8212;&#8212;&#8212;</i>
     </div>
     '''
m.get_root().html.add_child(folium.Element(legend_html))

# Save the map to an HTML file
m.save('figures/shortest_path_map_with_legend.html')

# Display the interactive map
m